In [3]:
from r_utils import *
from browser_libs import *
from mkdata import *
from cost_utils_libs import *

In [4]:
@memoize
def get_domain_to_session_id_to_seconds_for_install_id(install_id):
  output = {}
  seconds_on_domain_per_session = get_collection_for_install(install_id, 'synced:seconds_on_domain_per_session')
  for x in seconds_on_domain_per_session:
    domain = x['key']
    if domain not in output:
      output[domain] = {}
    session_id = x['key2']
    seconds = x['val']
    output[domain][session_id] = seconds
  return output

In [5]:
user_to_install_ids = get_user_to_all_install_ids()
info_list = []
for user in get_users():
  if user not in user_to_install_ids:
    continue
  #if len(user_to_install_ids[user]) != 1:
  #  continue
  impressions_info_list,actions_info_list,tab_id_to_session_id_to_impressions,tab_id_to_session_id_to_actions = get_impressions_paired_with_actions(user)
  #get_impressions_paired_with_actions(user)
  does_user_have_actions = False
  for x in actions_info_list:
    #print(x)
    #if not x['is_paired']:
    #  continue
    #if 'is_random' not in x:
    #  continue
    #if x['is_random'] == True:
    #  continue
    install_id = x['install_id']
    domain_to_session_id_to_seconds = get_domain_to_session_id_to_seconds_for_install_id(install_id)
    url = x['url']
    domain = url_to_domain(url)
    intervention = x['intervention']
    tab_id = x['tab_id']
    session_id = x['session_id']
    if (domain not in domain_to_session_id_to_seconds) or (session_id not in domain_to_session_id_to_seconds[domain]):
      continue
    seconds = domain_to_session_id_to_seconds[domain][session_id]
    have_prompt = False
    if x['is_paired'] and ('is_random' not in x):
      have_prompt = True
    info_list.append({'have_prompt': have_prompt, 'seconds': seconds, 'domain': domain, 'user': user, 'intervention': intervention})
    #action_timestamp = x['timestamp_local']
    #impression_info = tab_id_to_session_id_to_impressions[tab_id][session_id][0]
    #impression_timestamp = impression_info['timestamp_local']
    #latency = (action_timestamp - impression_timestamp) / 1000
    #if not (0 < latency < 30):
    #  continue
    #does_user_have_actions = True
    #latency_list_all.append(latency)
  #if does_user_have_actions:
  #  num_users += 1

KeyboardInterrupt: 